## DC level -Linear Regression

In [1]:
import pandas as pd
import numpy as np     
from sklearn.linear_model import LinearRegression
from fbprophet import Prophet
from pmdarima import auto_arima 
import statistics
# import matplotlib.pyplot as plt
# import seaborn as sns
import datetime as dt
from dateutil.relativedelta import relativedelta
import glob
import warnings 
import os

In [2]:


Current_month = dt.datetime.today().replace(minute=0, hour=0, second=0, microsecond=0,day=1) 
two_years = Current_month + relativedelta(years=2)
date = pd.DataFrame(pd.date_range(start=Current_month, end=two_years, freq='MS'),columns=['ds'])
path_IVsets_actuals=r'..\iv sets data for dashboard\IV sets'
path_pump_actuals=r"..\iv sets data for dashboard\IV pumps"
path_DC_State=r'..\iv sets data for dashboard\DC-state.xlsx'
path_pump_forecasted=r'..\iv sets data for dashboard\Forecasted Pumps.xlsx'
path_Output_pump=r'..\iv sets data for dashboard\IV Forecast new\Forecasting Using Pumps'
path_output_sarima_prophet=r'..\iv sets data for dashboard\IV Forecast new\hirunner Forecast'

def IVsets(path_DC_State,path_IVsets_actuals):
    try:
        IVsets = pd.DataFrame()
        for f in glob.glob(path_IVsets_actuals+"\*.xlsx"):
            df = pd.read_excel(f,usecols=['State','FY Month','Material#','Units'])
            IVsets = IVsets.append(df,ignore_index=True)

        DC_state=pd.read_excel(path_DC_State)
        IVsets['flag'] = [x if x in ['2420-0007','2420-0500','2426-0500','2426-0007'] else "aggregated" for x in IVsets['Material#'].tolist()]
        df_dc=pd.merge(IVsets,DC_state,how="left",left_on='State',right_on='State')
        df_dc['FY Month']=pd.to_datetime(df_dc['FY Month'])
        df_dc['Calendar Month']=pd.to_datetime(df_dc['FY Month'] - pd.DateOffset(months=3))
        df_dc['Calendar Month']=pd.to_datetime(df_dc['Calendar Month'])
        df_dc_agg=df_dc.groupby(['DC name','flag','Calendar Month'])['Units'].sum()
        df_dc_agg=pd.DataFrame(df_dc_agg)
        df_dc_agg.reset_index(inplace=True)

        df_dc_agg=df_dc_agg[df_dc_agg['Calendar Month']<Current_month]
        return df_dc_agg
    except Exception as e:
        print('Some error in IVsets function:'+str(e))
    

def PumpInstalledBase(path_pump_actuals,path_pump_forecasted):
    try:
        all_data = pd.DataFrame()
        for f in glob.glob(path_pump_actuals+"\*.xlsx"):
            df = pd.read_excel(f,usecols=['FY- Month','Act\nQty'])
            all_data = all_data.append(df,ignore_index=True)

        all_data['FY- Month']=pd.to_datetime(all_data['FY- Month'])
        all_data['Calendar Month']=all_data['FY- Month'] - pd.DateOffset(months=3)
        pump=all_data.groupby('Calendar Month',as_index=False)['Act\nQty'].sum().rename(columns={'Calendar Month':'Date','Act\nQty':'Units'}).sort_values('Date')

        pump_forecasted=pd.read_excel(path_pump_forecasted)
        pump_forecasted['Date']=pd.to_datetime(pump_forecasted['Date'])
        pump_forecasted=pump_forecasted[pump_forecasted['Date']>max(pump['Date'])].sort_values('Date')

        pump_total=pump.append(pump_forecasted).reset_index(drop=True)
        pump_total['retired_base']=pump_total['Units'].shift(108).fillna(0)
        pump_total['Actualunits']=pump_total["Units"]-pump_total['retired_base']   
        pump_total['Installed Base']=pump_total['Actualunits'].cumsum()
        pump_total=pump_total.set_index('Date')[['Installed Base']]
        return pump_total
    except Exception as e:
        print('Some error in PumpInstalledBase  function:'+str(e))

pump_installed_base=PumpInstalledBase(path_pump_actuals,path_pump_forecasted)
df_dc_agg=IVsets(path_DC_State,path_IVsets_actuals)


def model_pump(pump_installed_base,df_dc_agg):
    try:
        warnings.filterwarnings("ignore") 
        final=pd.DataFrame()
        high_runners=['2426-0007','2420-0007']
        for i in list(df_dc_agg['DC name'].unique()):
            df_redlands=df_dc_agg[df_dc_agg['DC name']==i]

            for j in high_runners:
                df_redlands=df_redlands.reset_index(drop=True)
                df_redlands2=df_redlands[df_redlands['flag']==j]
                df_2426=pd.DataFrame(df_redlands2.groupby('Calendar Month').Units.sum().rolling(9).mean())
                if j == '2426-0007':
                    a=7
                else:
                    a=13
                pump_installed_base['pump']=pump_installed_base['Installed Base'].shift(a)
                pump=pump_installed_base.drop('Installed Base',axis=1)
                data=pump.merge(df_2426,how='left',left_index=True,right_index=True)
                train=data.dropna()
                test=data[Current_month:]
                x1 = np.array(train.pump.values).reshape((-1, 1))
                y1 = np.array(train.Units.values)
                reg = LinearRegression().fit(x1,y1)
                offset=(train.Units[-1]/(reg.predict(np.array(test.pump.values).reshape((-1, 1))))[0])
                test['forecast'] = reg.predict(np.array(test.pump.values).reshape((-1, 1)))*offset
                final2=train.append(test)
                test.drop(['Units','pump'],axis=1,inplace=True)
                test['DC']=i
                test['material']=j
                final=final.append(test)

        return final

    except Exception as e:
        print('Some error in model function:'+str(e))
        
def model_sarima_prophet(df_dc_agg,date):
    try:
        warnings.filterwarnings("ignore") 
        final1=pd.DataFrame(columns=['ds','sku','dc','Sarima','Prophet'])

        DC=list(df_dc_agg['DC name'].unique())
        for DC_name in DC:
            df_redlands=df_dc_agg[df_dc_agg['DC name']==DC_name]
            df_redlands=df_redlands.reset_index(drop=True)
            monthly=list(df_redlands['flag'].unique())

            for my_time_series in monthly:
                x=df_redlands[df_redlands['flag']==my_time_series]
                x.set_index('Calendar Month',inplace=True)
                x=x.drop('flag',axis=1)
                date_df=pd.DataFrame(pd.date_range(start=df_redlands['Calendar Month'].min(), end=df_redlands['Calendar Month'].max(), freq='MS'),columns=['Calendar Month'])
                dummy=x.resample('MS').sum(min_count=1)
                dummy=pd.merge(dummy,date_df,how="right",on='Calendar Month')
                dummy=dummy.fillna(0)
                dummy.sort_values('Calendar Month',inplace=True)
                dummy=dummy.rename(columns={'Calendar Month':"ds",'Units':'y'}).reset_index().drop('index',axis=1)

                ######################################################################################################################
                                                                # PROPHET
                m = Prophet()
                m.fit(dummy)
                y_pred = pd.DataFrame(m.predict(date))
                pred=pd.DataFrame(y_pred.yhat).set_index(date.ds)

                ######################################################################################################################
                                                                # SARIMA

                model = auto_arima(dummy['y'], start_p = 1, start_q = 1, 
                                      max_p = 3, max_q = 3, 
                                      seasonal = True, 
                                      d = None, D = 1, trace = True, 
                                      error_action ='ignore',   
                                      suppress_warnings = True,  
                                      stepwise = True) 
                y_pred_sar = model.predict(n_periods=len(date))
                pred_sar = pd.DataFrame(y_pred_sar,index = date.ds,columns=['Prediction'])

                final=pd.DataFrame()
                final['ds']=date.ds
                final['Sarima']=pred_sar.reset_index().Prediction
                final['Prophet']=pred.reset_index().yhat
                final['sku']=my_time_series
                final['dc']=DC_name
                final1=final1.append(final)
        return final1         
    except Exception as e:
        print('Some error in model function:'+str(e))
            


In [3]:
final_pump=model_pump(pump_installed_base,df_dc_agg)
final_pump=final_pump.reset_index()
final_pump=final_pump[final_pump['Date']<=two_years]
final_pump['Date']=final_pump['Date'].dt.strftime('%m/%d/%Y')
final_pump['Snapshot']=pd.to_datetime(Current_month, format='%Y-%m-%d', errors='ignore')
final_pump['Snapshot']=final_pump['Snapshot'].dt.strftime('%m/%d/%Y')
final_pump=final_pump.rename(columns={'Date':'Calender_date', 'forecast':'forecasted', 'DC':'DC_forecast', 'material':'Material#'})

In [4]:
final_pump

,Calender_date,forecasted,DC_forecast,Material#,Snapshot
0,03/01/2022,473988.888889,Four Oaks,2426-0007,03/01/2022
1,04/01/2022,471576.900237,Four Oaks,2426-0007,03/01/2022
2,05/01/2022,468873.729564,Four Oaks,2426-0007,03/01/2022
3,06/01/2022,467167.440015,Four Oaks,2426-0007,03/01/2022
4,07/01/2022,463000.940587,Four Oaks,2426-0007,03/01/2022
...,...,...,...,...,...
175,11/01/2023,776131.335080,Redlands,2420-0007,03/01/2022
176,12/01/2023,773067.702088,Redlands,2420-0007,03/01/2022
177,01/01/2024,756393.319109,Redlands,2420-0007,03/01/2022
178,02/01/2024,757871.584297,Redlands,2420-0007,03/01/2022


In [5]:
final_pump.to_excel(path_Output_pump+'\Forecast_usingIB_Snapshot_'+Current_month.strftime("%m-%d-%Y")+'.xlsx',index=False)

In [6]:
final_sarima_prophet=model_sarima_prophet(df_dc_agg,date)

INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


Performing stepwise search to minimize aic
 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=3685.351, Time=0.14 sec
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=3749.536, Time=0.00 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=3711.558, Time=0.03 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=3697.622, Time=0.03 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=3748.060, Time=0.01 sec
 ARIMA(2,1,1)(0,0,0)[0] intercept   : AIC=3672.426, Time=0.05 sec
 ARIMA(2,1,0)(0,0,0)[0] intercept   : AIC=3684.074, Time=0.02 sec
 ARIMA(3,1,1)(0,0,0)[0] intercept   : AIC=3672.400, Time=0.07 sec
 ARIMA(3,1,0)(0,0,0)[0] intercept   : AIC=3682.629, Time=0.03 sec
 ARIMA(3,1,2)(0,0,0)[0] intercept   : AIC=inf, Time=0.37 sec
 ARIMA(2,1,2)(0,0,0)[0] intercept   : AIC=3687.244, Time=0.12 sec
 ARIMA(3,1,1)(0,0,0)[0]             : AIC=3682.759, Time=0.05 sec

Best model:  ARIMA(3,1,1)(0,0,0)[0] intercept
Total fit time: 0.949 seconds


INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


Performing stepwise search to minimize aic
 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=3623.547, Time=0.04 sec
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=3703.836, Time=0.01 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=3671.660, Time=0.03 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=3622.192, Time=0.03 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=3701.840, Time=0.01 sec
 ARIMA(0,1,2)(0,0,0)[0] intercept   : AIC=3676.355, Time=0.06 sec
 ARIMA(1,1,2)(0,0,0)[0] intercept   : AIC=3665.773, Time=0.23 sec
 ARIMA(0,1,1)(0,0,0)[0]             : AIC=3621.167, Time=0.02 sec
 ARIMA(1,1,1)(0,0,0)[0]             : AIC=3622.550, Time=0.05 sec
 ARIMA(0,1,2)(0,0,0)[0]             : AIC=3653.300, Time=0.03 sec
 ARIMA(1,1,0)(0,0,0)[0]             : AIC=3669.660, Time=0.02 sec
 ARIMA(1,1,2)(0,0,0)[0]             : AIC=3646.713, Time=0.08 sec

Best model:  ARIMA(0,1,1)(0,0,0)[0]          
Total fit time: 0.620 seconds


INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


...

 ARIMA(1,0,2)(0,0,0)[0] intercept   : AIC=3338.852, Time=0.08 sec
 ARIMA(0,0,1)(0,0,0)[0]             : AIC=3665.564, Time=0.02 sec

Best model:  ARIMA(0,0,1)(0,0,0)[0] intercept
Total fit time: 0.256 seconds
Performing stepwise search to minimize aic
 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=3593.920, Time=0.03 sec
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=3677.322, Time=0.01 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=3633.485, Time=0.01 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=3601.557, Time=0.04 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=3675.409, Time=0.01 sec
 ARIMA(2,1,1)(0,0,0)[0] intercept   : AIC=3588.655, Time=0.05 sec
 ARIMA(2,1,0)(0,0,0)[0] intercept   : AIC=3611.612, Time=0.03 sec
 ARIMA(3,1,1)(0,0,0)[0] intercept   : AIC=3587.108, Time=0.07 sec
 ARIMA(3,1,0)(0,0,0)[0] intercept   : AIC=3597.672, Time=0.04 sec
 ARIMA(3,1,2)(0,0,0)[0] intercept   : AIC=3592.455, Time=0.14 sec


INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


 ARIMA(2,1,2)(0,0,0)[0] intercept   : AIC=inf, Time=0.33 sec
 ARIMA(3,1,1)(0,0,0)[0]             : AIC=3590.079, Time=0.06 sec

Best model:  ARIMA(3,1,1)(0,0,0)[0] intercept
Total fit time: 0.837 seconds
Performing stepwise search to minimize aic
 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=3366.457, Time=0.03 sec
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=3439.133, Time=0.01 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=3409.576, Time=0.02 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=3364.729, Time=0.02 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=3437.153, Time=0.01 sec
 ARIMA(0,1,2)(0,0,0)[0] intercept   : AIC=3401.320, Time=0.03 sec
 ARIMA(1,1,2)(0,0,0)[0] intercept   : AIC=inf, Time=0.15 sec
 ARIMA(0,1,1)(0,0,0)[0]             : AIC=3363.711, Time=0.01 sec
 ARIMA(1,1,1)(0,0,0)[0]             : AIC=3365.253, Time=0.05 sec
 ARIMA(0,1,2)(0,0,0)[0]             : AIC=3371.602, Time=0.03 sec
 ARIMA(1,1,0)(0,0,0)[0]             : AIC=3407.653, Time=0.02 sec
 ARIMA(1,1,2)(0,0,0)[0]         

INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


Performing stepwise search to minimize aic
 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=3526.180, Time=0.05 sec
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=3586.737, Time=0.01 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=3566.740, Time=0.03 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=3524.430, Time=0.04 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=3584.749, Time=0.00 sec
 ARIMA(0,1,2)(0,0,0)[0] intercept   : AIC=3559.308, Time=0.03 sec
 ARIMA(1,1,2)(0,0,0)[0] intercept   : AIC=inf, Time=0.26 sec
 ARIMA(0,1,1)(0,0,0)[0]             : AIC=3518.603, Time=0.01 sec
 ARIMA(1,1,1)(0,0,0)[0]             : AIC=3519.828, Time=0.05 sec
 ARIMA(0,1,2)(0,0,0)[0]             : AIC=3553.058, Time=0.04 sec
 ARIMA(1,1,0)(0,0,0)[0]             : AIC=3564.719, Time=0.02 sec
 ARIMA(1,1,2)(0,0,0)[0]             : AIC=3555.672, Time=0.11 sec

Best model:  ARIMA(0,1,1)(0,0,0)[0]          
Total fit time: 0.666 seconds


In [7]:
final_sarima_prophet.rename(columns={'ds':'Calender_date','sku':'Material#','dc':'DC'},inplace=True)

In [8]:
final_sarima_prophet['Snapshot']=pd.to_datetime(Current_month, format='%Y-%m-%d', errors='ignore').strftime('%m/%d/%Y')
final_sarima_prophet['Calender_date']=final_sarima_prophet['Calender_date'].dt.strftime('%m/%d/%Y')

In [9]:
final_sarima_prophet

,Calender_date,Material#,DC,Sarima,Prophet,Snapshot
0,03/01/2022,2420-0007,Four Oaks,2.401696e+06,2.369427e+06,03/01/2022
1,04/01/2022,2420-0007,Four Oaks,2.404602e+06,2.474497e+06,03/01/2022
2,05/01/2022,2420-0007,Four Oaks,2.409483e+06,2.356277e+06,03/01/2022
3,06/01/2022,2420-0007,Four Oaks,2.422453e+06,2.519550e+06,03/01/2022
4,07/01/2022,2420-0007,Four Oaks,2.453453e+06,2.400223e+06,03/01/2022
...,...,...,...,...,...,...
20,11/01/2023,aggregated,Redlands,4.594864e+05,4.705219e+05,03/01/2022
21,12/01/2023,aggregated,Redlands,4.594864e+05,4.861127e+05,03/01/2022
22,01/01/2024,aggregated,Redlands,4.594864e+05,4.819784e+05,03/01/2022
23,02/01/2024,aggregated,Redlands,4.594864e+05,4.630643e+05,03/01/2022


In [10]:
#final_sarima_prophet.to_excel(path_output_sarima_prophet+'\highrunner_forecastsnapshot_'+Current_month.strftime("%m-%d-%Y")+'.xlsx',index=False)